## Noise-to-Aggregate-Ratio (NAR)

In [2]:
import nilmtk
print(nilmtk.__version__)

0.4.0.dev1+git.


### NAR: What is the noise level in a dataset?


The aggregate power signal of a real-world dataset consists not exclusively of known appliance-level signals, but also contains several unknown appliance-level signals that contribute to the error term epsilon.

 The Noise-to-Aggregate Ratio, NAR, is defined as:

![NAR](img/NAR.png)

the ratio between noise and aggregate signal over a time window T,  where xi is the power consumption of appliance i, y the aggregate power signal, and T the length of the observed time frame. The noise-to-aggregate ratio (NAR) can be computed for all AC power types, as long as energy readings of aggregate and submeters are available.
A ratio of 0.25 reports that 25\% of the total energy consumption stems from unmetered appliances and noise. Hence, the ratio indicates to what degree information on the aggregate's components is available.

#### Implementation

Our source code builds on functions provided by [NILMTK](https://github.com/nilmtk/nilmtk) to foster reproducibility of results.
Our implementation of the NAR can be obtained from *NAR.py* and defines the following function:

```python
from NAR import noise_aggregate_ratio

NAR_p = noise_aggregate_ratio(elec, power_type='active')
NAR_s = noise_aggregate_ratio(elec, power_type='apparent')
```

with the following input parameters:

* *elec_meter:* MeterGroup object of a household or a simple [MeterGroup](http://nilmtk.github.io/nilmtk/master/_modules/nilmtk/metergroup.html)
* *power_type:* String that defines the AC power type of interest: active or apparent

## Usage Notes

We will demonstrate the use of NAR by the help of a simple example. First, we have to load the dataset using NILMTK

In [4]:
from nilmtk import DataSet
from NAR import noise_aggregate_ratio

d_dir = '/Users/christoph/datasets/'
dataset = ['iAWE', 1]

dset = DataSet(d_dir+'{}.h5'.format(dataset[0]))
household = dataset[1]
elec = dset.buildings[household].elec
print(elec)

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=3, building=1, dataset='iAWE', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=4, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=5, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=2)])
  ElecMeter(instance=6, building=1, dataset='iAWE', appliances=[Appliance(type='washing machine', instance=1)])
  ElecMeter(instance=7, building=1, dataset='iAWE', appliances=[Appliance(type='computer', instance=1)])
  ElecMeter(instance=8, building=1, dataset='iAWE', appliances=[Appliance(type='clothes iron', instance=1)])
  ElecMeter(instance=9, building=1, dataset='iAWE', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=10, building=1, dataset='iAWE', appliances=[A

Now, we can compute the NAR's for active and apprent power of iAWE:

In [10]:
NAR_p = 100 * noise_aggregate_ratio(elec, power_type='active')
print('{} %'.format(NAR_p))
NAR_s = 100 * noise_aggregate_ratio(elec, power_type='apparent')
print('{} %'.format(NAR_s))


Calculating total_energy for ElecMeterID(instance=2, building=1, dataset='iAWE') ...   63.0 %
Calculating total_energy for ElecMeterID(instance=2, building=1, dataset='iAWE') ...   61.0 %


As we see, house 1 of iAWE has a NAR of 63 % for active power and a NAR of 61 % for apparent power.

#### Why should you use NAR and not proportion-of-energy?

NILMTK includes a function called [*proportion_of_energy(self, other)*](http://nilmtk.github.io/nilmtk/master/_modules/nilmtk/electric.html#Electric.proportion_of_energy) that was introduced to report the amount of energy consumed by one appliance. However, we found some fundamental problems with that function, which must not be ignored.

The first problem with this function is that it mixes active and apparent power during computation, which must not be done in any situation.

```python
elif n_shared_ac_types == 0:
            ac_type = select_best_ac_type(self_ac_types)
            other_ac_type = select_best_ac_type(other_ac_types)
...
return total_energy[ac_type] / other_total_energy[other_ac_type]

```
The second problem with this function is that it returns the average proportion of energy for several power types.
```python
if n_shared_ac_types > 1:
            return (total_energy[shared_ac_types] /other_total_energy[shared_ac_types]).mean() 
```


### NAR in common LF energy consumption data sets

We derived the NAR with respect to active and apparent power for some of the most commonly-used energy consumption data sets. Please note that our focus is on their *low-frequency* version.

| Data Set | House | Duration in days | Meters | NAR for P in \%| NAR for S in \%|
|----------|-------:|------------------:|--------:|-----------:|-----------:|
| AMPds2   |  1    | 730              |   20     |     18      |   6       |
| COMBED   |  1    |  28              |   13    |    34        |    -    |
| DRED     |  1    |  153             |   12     |   -       |     28    |
| ECO      |  1    |  245             |   7     |    68        |     -      |
| ECO      |  6    |  219             |   7     |    74        |   -        |
| iAWE     |  1    |  73              |   10     |   63        |     61      |
| REDD     |  1    |  36                |   16     |  -     |    -        |
| REFIT    |  1    |  638              |    9    |     65      |   -       |
| REFIT    |  8    |  555                |  9      |   78      |    -        |
| REFIT    |  17   |  443             |     9   |      45      |   -       |
| UK-DALE  |  1    |  658              |    52    |    33      |    87       |
| UK-DALE  |  2    |  176             |     18   |     41      |  -       |
| UK-DALE  |  5    |  137             |     24   |     31      |   -       |
